In [1]:
# !pip install wget
!pip install torch -q
!pip install transformers -q
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00


In [2]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  device_count = torch.cuda.device_count()
  device_name = torch.cuda.get_device_name(0)

  print(f"There are {device_count} GPU(s) available.")
  print(f"We will use the GPU: {device_name}")


else:
  print("No GPU available, using the CPU instead.")
  device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
import pandas as pd
from datasets import load_dataset

dataset = None
dataset = load_dataset('glue', 'cola')


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [4]:
df_train = dataset['train'].to_pandas()
df_train = df_train.sample(1000)

# Report the number of sentences
print(f"Number of training sentences are: {df_train.shape[0]}")

# Display 10 random rows from the data
df_train.sample(10)

Number of training sentences are: 1000


,sentence,label,idx
8184,The librarian likes books.,1,8184
1459,It is not true that that Bob was lying was obv...,1,1459
4646,Chris was handed a note.,1,4646
3553,He treats John very shame.,0,3553
773,Our love they.,1,773
4231,Everyone of us be given a prize.,1,4231
5285,How likely to win is he?,1,5285
6503,Every student answered any question.,0,6503
5500,You have presented so elegant a solution that ...,1,5500
4476,Don't be silly!,1,4476


In [5]:
sentences = df_train.sentence.values
labels = df_train.label.values

print(f"Shape of Sentences are: {sentences.shape}")
print(f"Shape of Labels are: {labels.shape}")

Shape of Sentences are: (1000,)
Shape of Labels are: (1000,)


In [6]:
df_train.head()

,sentence,label,idx
6135,The bed was fallen on by the dust.,0,6135
2582,Water poured onto the plants.,1,2582
4209,Most of the milk your baby consumes during bre...,1,4209
8125,Gillian has made pasta and David is too.,0,8125
6212,Tami is likely to leave New York.,1,6212


In [7]:
from transformers import BertTokenizer

print("Loading BERT tokenizer....")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loading BERT tokenizer....


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Display the original sentence
print(f"Original: {sentences[0]}")

# Tokenization
print(f'Tokenized: {tokenizer.tokenize(sentences[0])}')

# Print the sentence mapped to token ids.
print(f'Token IDs: {tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0]))}')

Original: The bed was fallen on by the dust.
Tokenized: ['the', 'bed', 'was', 'fallen', 'on', 'by', 'the', 'dust', '.']
Token IDs: [1996, 2793, 2001, 5357, 2006, 2011, 1996, 6497, 1012]


In [9]:

max_len = 0

for sentence in sentences:

  # Tokenize the text and add `[CLS]` and `[SEP]` tokens
  input_ids = tokenizer.encode(sentence, add_special_tokens=True)

  max_len = max(max_len, len(input_ids))

print(f"Max sentence length: {max_len}")

Max sentence length: 29


In [10]:
input_ids = []
attention_masks = []

for sentence in sentences:
  # tokenizer.encode_plus() includes all the abstractions except adding
  # attention masks
  encoded_dict = tokenizer.encode_plus(
      sentence,
      add_special_tokens = True,
      max_length = 64,
      pad_to_max_length = True,
      return_attention_mask=True,
      return_tensors='pt'
  )

  # Add the encoded sentence to the list
  input_ids.append(encoded_dict['input_ids'])

  # Add the attention mask to the list
  attention_masks.append(encoded_dict['attention_mask'])

# Conver the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
# Display sentence 0
print(f'Original: {sentences[0]}')
print(f'Tokens Ids: {input_ids[0]}')

Original: The bed was fallen on by the dust.
Tokens Ids: tensor([ 101, 1996, 2793, 2001, 5357, 2006, 2011, 1996, 6497, 1012,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


In [12]:
from torch.utils.data import TensorDataset, random_split

# combine the training inputs into a TensorDataset
dataset_tensor = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.9 * len(dataset_tensor))
val_size = len(dataset_tensor) - train_size

# Random Selection
train_dataset, val_dataset = random_split(dataset_tensor, [train_size, val_size])

print(f"Training samples: {train_size}")
print(f"Validation samples: {val_size}")

Training samples: 900
Validation samples: 100


In [13]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, AdamW
from torch.utils.data import Dataset, DataLoader

# Custom dataset
class PoetryDataset(Dataset):
    def __init__(self, sentences, poems, tokenizer, max_length):
        self.sentences = sentences
        self.poems = poems
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        poem = self.poems[idx]

        inputs = self.tokenizer(sentence, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        labels = self.tokenizer(poem, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')['input_ids']

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

# Fine-tuning function
def fine_tune_bert_for_poetry(sentences, poems, model_name='bert-base-uncased', epochs=3, batch_size=8):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForMaskedLM.from_pretrained(model_name)

    dataset = PoetryDataset(sentences, poems, tokenizer, max_length=64)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = AdamW(model.parameters(), lr=5e-5)

    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
            loss = outputs.loss
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs} completed")

    return model, tokenizer

# Generate poetry function
def generate_poetry(model, tokenizer, sentence, max_length=64):
    input_ids = tokenizer.encode(sentence, return_tensors='pt', max_length=max_length, truncation=True)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
sentences = ["The sun sets in the west", "A gentle breeze blows", "The stars twinkle at night"]
poems = ["Golden orb descends, / Painting skies with vibrant hues, / Day bids farewell, peace.",
         "Soft whispers of air, / Caressing leaves and flowers, / Nature's lullaby.",
         "Celestial diamonds, / Adorning the velvet night, / Silent cosmic dance."]

model, tokenizer = fine_tune_bert_for_poetry(sentences, poems)

# Generate a poem from a new sentence
new_sentence = "The river flows swiftly"
generated_poem = generate_poetry(model, tokenizer, new_sentence)
print(f"Input: {new_sentence}")
print(f"Generated poem: {generated_poem}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3 completed
Epoch 2/3 completed
Epoch 3/3 completed
Input: The river flows swiftly
Generated poem: the river flows swiftly downstream downstream upstream and upstream upstream downstream tributaries upstream tributaries downstream and downstream river tributaries tributaries and tributaries river downstream headwaters downstream confluence downstream or upstream river upstream confluence upstream headwaters upstream dams downstream dams upstream or downstream runoff downstream rivers downstream / downstream navigable downstreamriver downstream waterways downstream basins downstream projects downstream streams upstream rivers upstream


In [14]:
sentence = "Ramesh is gay"
generated_poem = generate_poetry(model, tokenizer, sentence)

In [16]:
print(generated_poem)

ramesh is gay india india pakistan nepal nepal pakistan india nepal india bangladesh pakistan bangladesh nepal bangladesh bangladesh bhutan bangladesh india bhutan nepal bhutan bhutan pakistan pakistan bhutan india nepali nepal nepali bhutan nepali nepali bangladesh kathmandu nepal kathmandu kathmandu bhutan kathmandu nepali kathmandu bangladesh nepali tibet nepal tibet bhutan tibet kathmandu tibet nepali cambodia kathmandu cambodia nepal cambodia bhutan cambodia
